The following code uses data acquired on April 22, 2021 from the World Health Organization (WHO) and the Center for Disease Control (CDC). The purpose of this project is to establish the current trends of the COVID-19 disease in relation to the world as a whole and in just the United States.

In [1]:
#importing modules
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
#load data

#USA deaths by week w/ comorbidity by state
cdc_deaths_raw = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\USA_DEATHS_BY_WEEK.csv')

#Worldwide deaths and case numbers
who_raw = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\WHO-COVID-19-global-data.csv')

In [3]:
#USA gdp by state (source BEA)
bea_gdp_raw = pd.read_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\GDP_per_state.csv', error_bad_lines=False)

#USA voting and obesity statistics by state (sources National Archives and CDC) 
cdc_VO_raw = pd.read_excel(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\State_voting_and_obesity.xlsx')

#USA percentage of urban populace by state. Definition of 'Urban': included all population in urbanized areas
#and urban clusters (each with their own population size and density thresholds). (source census.gov)
cdc_urb_raw = pd.read_excel(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\urban_pop_clean.xlsx')


First we need to clean the data from the WHO

In [4]:
who_raw.tail()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
113044,2021-04-19,ZW,Zimbabwe,AFRO,82,37751,1,1553
113045,2021-04-20,ZW,Zimbabwe,AFRO,108,37859,0,1553
113046,2021-04-21,ZW,Zimbabwe,AFRO,16,37875,1,1554
113047,2021-04-22,ZW,Zimbabwe,AFRO,105,37980,1,1555
113048,2021-04-23,ZW,Zimbabwe,AFRO,38,38018,0,1555


In [5]:
who_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113049 entries, 0 to 113048
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      113049 non-null  object
 1   Country_code       112572 non-null  object
 2   Country            113049 non-null  object
 3   WHO_region         113049 non-null  object
 4   New_cases          113049 non-null  int64 
 5   Cumulative_cases   113049 non-null  int64 
 6   New_deaths         113049 non-null  int64 
 7   Cumulative_deaths  113049 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 6.9+ MB


Here we see some null values in Country Code. We'll have to deal with that.

In [6]:
who_raw.describe()

,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
count,113049.000000,1.130490e+05,113049.000000,113049.000000
mean,1276.959159,1.800028e+05,27.121983,4422.662907
std,8014.842116,1.172761e+06,152.176415,24146.188185
min,-32952.000000,0.000000e+00,-514.000000,0.000000
25%,0.000000,1.300000e+01,0.000000,0.000000
50%,5.000000,1.650000e+03,0.000000,27.000000
75%,248.000000,3.165800e+04,4.000000,561.000000
max,402270.000000,3.153021e+07,6409.000000,564091.000000


We have negative values for both New_cases and New_deaths. That is obviously not possible. From the WHO literature, though, we know that those columns are calculated based on reported cumulative number from one day minus the cumulative number of the next. We must be getting mis-reported numbers of cases and deaths sometimes. We'll check out the worst of those cases and see if we should do something about it.

In [7]:
who_raw[who_raw.New_cases < 0]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
77,2020-03-20,AF,Afghanistan,EMRO,-2,24,0,0
2483,2020-04-10,AO,Angola,AFRO,-2,17,0,2
2955,2020-04-05,AI,Anguilla,AMRO,-1,2,0,0
3420,2020-03-24,AG,Antigua and Barbuda,AMRO,-2,3,0,0
4846,2020-03-19,AW,Aruba,AMRO,-3,5,0,0
...,...,...,...,...,...,...,...,...
108557,2020-10-07,VI,United States Virgin Islands,AMRO,-5,1322,0,20
108558,2020-10-08,VI,United States Virgin Islands,AMRO,-1,1321,0,20
108566,2020-10-16,VI,United States Virgin Islands,AMRO,-1,1327,0,20
110686,2020-01-25,VN,Viet Nam,WPRO,-4,2,0,0


Only 88 rows out of over 113,000. Not bad, let's drop them and put them into our modified data frame we'll just call 'who'

In [8]:
who = who_raw[who_raw.New_cases >= 0]

In [9]:
who_raw[who_raw.New_deaths < 0].shape

(36, 8)

Only 36 rows for misreported deaths. Let's drop them too.

In [10]:
who = who[who.New_deaths >= 0]

In [11]:
who.describe()

,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
count,112927.000000,1.129270e+05,112927.000000,112927.000000
mean,1278.481922,1.801754e+05,27.121264,4425.724973
std,8017.913969,1.173378e+06,151.832463,24157.657554
min,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000,1.300000e+01,0.000000,0.000000
50%,5.000000,1.655000e+03,0.000000,27.000000
75%,249.000000,3.177500e+04,4.000000,561.000000
max,402270.000000,3.153021e+07,6409.000000,564091.000000


That looks far better.

In [12]:
who.Country_code.unique()

array(['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW',
       'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ',
       'BM', 'BT', 'BO', 'XA', 'BA', 'BW', 'BR', 'VG', 'BN', 'BG', 'BF',
       'BI', 'CV', 'KH', 'CM', 'CA', 'KY', 'CF', 'TD', 'CL', 'CN', 'CO',
       'KM', 'CG', 'CK', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'KP',
       'CD', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE',
       'SZ', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'GA', 'GM',
       'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG',
       'GN', 'GW', 'GY', 'HT', 'VA', 'HN', 'HU', 'IS', 'IN', 'ID', 'IR',
       'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE',
       'KI', 'XK', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI',
       'LT', 'LU', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR',
       'MU', 'YT', 'MX', 'FM', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM',
       nan, 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE

We have a blank space and a nan space in our country codes. Let's figure out what those are.

In [13]:
who[who.Country_code == ' ']

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
75366,2020-01-03,,Other,Other,0,0,0,0
75367,2020-01-04,,Other,Other,0,0,0,0
75368,2020-01-05,,Other,Other,0,0,0,0
75369,2020-01-06,,Other,Other,0,0,0,0
75370,2020-01-07,,Other,Other,0,0,0,0
...,...,...,...,...,...,...,...,...
75838,2021-04-19,,Other,Other,0,745,0,13
75839,2021-04-20,,Other,Other,0,745,0,13
75840,2021-04-21,,Other,Other,0,745,0,13
75841,2021-04-22,,Other,Other,0,745,0,13


It's unclear where these are from. There are also few enough of them that it shouldn't matter too much if we remove them.

In [14]:
who = who[who.Country_code != ' ']

In [15]:
who[who.Country_code.isna()]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
68211,2020-01-03,NaN,Namibia,AFRO,0,0,0,0
68212,2020-01-04,NaN,Namibia,AFRO,0,0,0,0
68213,2020-01-05,NaN,Namibia,AFRO,0,0,0,0
68214,2020-01-06,NaN,Namibia,AFRO,0,0,0,0
68215,2020-01-07,NaN,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
68683,2021-04-19,NaN,Namibia,AFRO,140,46655,2,604
68684,2021-04-20,NaN,Namibia,AFRO,118,46773,0,604
68685,2021-04-21,NaN,Namibia,AFRO,102,46875,4,608
68686,2021-04-22,NaN,Namibia,AFRO,79,46954,1,609


It looks like the NaN's are all from Namibia. This could be explained when we realize that the country code for Namibia is literally 'NA'. Let's fill those NaN's with with just the letters 'NA'.

In [16]:
who.fillna('NA', inplace=True)
who.info(), who.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112451 entries, 0 to 113048
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      112451 non-null  object
 1   Country_code       112451 non-null  object
 2   Country            112451 non-null  object
 3   WHO_region         112451 non-null  object
 4   New_cases          112451 non-null  int64 
 5   Cumulative_cases   112451 non-null  int64 
 6   New_deaths         112451 non-null  int64 
 7   Cumulative_deaths  112451 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 7.7+ MB


(None,
            New_cases  Cumulative_cases     New_deaths  Cumulative_deaths
 count  112451.000000      1.124510e+05  112451.000000      112451.000000
 mean     1283.887017      1.809352e+05      27.235961        4444.411904
 std      8034.434583      1.175800e+06     152.143219       24207.022016
 min         0.000000      0.000000e+00       0.000000           0.000000
 25%         0.000000      1.300000e+01       0.000000           0.000000
 50%         6.000000      1.703000e+03       0.000000          28.000000
 75%       252.500000      3.236400e+04       4.000000         570.000000
 max    402270.000000      3.153021e+07    6409.000000      564091.000000)

Looking far better. Though, 402,270 new cases in a single day looks a little high. Let's double check that.

In [17]:
who[who.New_cases > 350000]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
108154,2020-12-20,US,United States of America,AMRO,402270,17314834,2747,311150


Looks like it comes from the US. Let's take a look at how the US is taking the virus in general and see if 402,270 new cases in a single day is extrememly out of the ordinary.

In [18]:
who.loc[108145:108160, :]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
108145,2020-12-11,US,United States of America,AMRO,230852,15203208,3390,287384
108146,2020-12-12,US,United States of America,AMRO,201681,15404889,2749,290133
108147,2020-12-13,US,United States of America,AMRO,243209,15648098,2996,293129
108148,2020-12-14,US,United States of America,AMRO,212577,15860675,2277,295406
108149,2020-12-15,US,United States of America,AMRO,180420,16041095,1434,296840
108150,2020-12-16,US,United States of America,AMRO,204281,16245376,1754,298594
108151,2020-12-17,US,United States of America,AMRO,201468,16446844,2942,301536
108152,2020-12-18,US,United States of America,AMRO,235805,16682649,3424,304960
108153,2020-12-19,US,United States of America,AMRO,229915,16912564,3443,308403
108154,2020-12-20,US,United States of America,AMRO,402270,17314834,2747,311150


It looks like that day had nearly twice as many new cases as the days around it. This seems hard to believe and doesn't align with current literature so we'll remove it.

In [19]:
who = who[who.New_cases < 350000]
who[who.New_cases > 350000]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths


Now let's clean up the USA state data and combine it into one dataframe with multiple variables

In [20]:
cdc_deaths_raw.head()

,Data as of,Start Date,End Date,Group,Year,Month,MMWR Week,Week Ending Date,State,COVID-19 Deaths,Total Deaths,Percent of Expected Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
0,04/23/2021,12/29/2019,01/04/2020,By Week,2019/2020,NaN,1.0,01/04/2020,United States,0.0,60163.0,98.0,4105.0,0.0,434.0,4539.0,NaN
1,04/23/2021,01/05/2020,01/11/2020,By Week,2020,NaN,2.0,01/11/2020,United States,0.0,60724.0,97.0,4153.0,0.0,473.0,4626.0,NaN
2,04/23/2021,01/12/2020,01/18/2020,By Week,2020,NaN,3.0,01/18/2020,United States,3.0,59352.0,97.0,4064.0,3.0,467.0,4531.0,NaN
3,04/23/2021,01/19/2020,01/25/2020,By Week,2020,NaN,4.0,01/25/2020,United States,1.0,59152.0,99.0,3917.0,0.0,499.0,4417.0,NaN
4,04/23/2021,01/26/2020,02/01/2020,By Week,2020,NaN,5.0,02/01/2020,United States,0.0,58823.0,99.0,3817.0,0.0,481.0,4298.0,NaN


In [21]:
cdc_deaths_raw['Data as of'].unique()

array(['04/23/2021'], dtype=object)

Useful to know, but useless to our analysis. Let's remove this column and start building our main DataFrame. We'll call it the classic 'df'.

In [22]:
df = cdc_deaths_raw.drop('Data as of', axis=1)
df.head()

,Start Date,End Date,Group,Year,Month,MMWR Week,Week Ending Date,State,COVID-19 Deaths,Total Deaths,Percent of Expected Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
0,12/29/2019,01/04/2020,By Week,2019/2020,NaN,1.0,01/04/2020,United States,0.0,60163.0,98.0,4105.0,0.0,434.0,4539.0,NaN
1,01/05/2020,01/11/2020,By Week,2020,NaN,2.0,01/11/2020,United States,0.0,60724.0,97.0,4153.0,0.0,473.0,4626.0,NaN
2,01/12/2020,01/18/2020,By Week,2020,NaN,3.0,01/18/2020,United States,3.0,59352.0,97.0,4064.0,3.0,467.0,4531.0,NaN
3,01/19/2020,01/25/2020,By Week,2020,NaN,4.0,01/25/2020,United States,1.0,59152.0,99.0,3917.0,0.0,499.0,4417.0,NaN
4,01/26/2020,02/01/2020,By Week,2020,NaN,5.0,02/01/2020,United States,0.0,58823.0,99.0,3817.0,0.0,481.0,4298.0,NaN


We're interested in only the number of covid cases and deaths per week. The other information here is interesting but outside the scope of this study. Let's remove all but the essential columns. We'll discern week by the 'End Date' column. First, let's only keep the information that where Group == 'By Week'.

In [23]:
df = df[df.Group == 'By Week']
df.Group.unique()

array(['By Week'], dtype=object)

It's important to realize that 'Total Deaths' are not COVID related deaths. They instead just refer to the number of people who died from any cause in the USA during that time period. 'Percent of Expected Deaths' just shows if the number of deaths in the USA was as expected in a normal year or if it has had an impact from COVID. We are just interested in COVID-19 deaths so will remove nearly all of these columns.

We'll add one more column that counts the total number of COVID related deaths per state.

In [24]:
df = df[['Week Ending Date', 'COVID-19 Deaths', 'State']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 0 to 3671
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Week Ending Date  3672 non-null   object 
 1   COVID-19 Deaths   3236 non-null   float64
 2   State             3672 non-null   object 
dtypes: float64(1), object(2)
memory usage: 114.8+ KB


We have some null values do to suppression in accordance with NCHS confidentiality standards. As such, we shall fill with 0's. The total deaths per state have been cross-referenced with current literature and they align. The number of patients suppressed should be minimal.

In [25]:
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 0 to 3671
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Week Ending Date  3672 non-null   object 
 1   COVID-19 Deaths   3672 non-null   float64
 2   State             3672 non-null   object 
dtypes: float64(1), object(2)
memory usage: 114.8+ KB


In [26]:
df_fin = pd.DataFrame()
for state_name in df.State.unique():
    x = df[df.State == state_name]
    x['total_covid_deaths'] = x['COVID-19 Deaths'].cumsum()
    df_fin = df_fin.append(x)
df = df_fin
df

<ipython-input-26-565cc5e346c2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['total_covid_deaths'] = x['COVID-19 Deaths'].cumsum()


,Week Ending Date,COVID-19 Deaths,State,total_covid_deaths
0,01/04/2020,0.0,United States,0.0
1,01/11/2020,0.0,United States,0.0
2,01/18/2020,3.0,United States,3.0
3,01/25/2020,1.0,United States,4.0
4,02/01/2020,0.0,United States,4.0
...,...,...,...,...
3667,03/20/2021,18.0,Puerto Rico,1968.0
3668,03/27/2021,13.0,Puerto Rico,1981.0
3669,04/03/2021,15.0,Puerto Rico,1996.0
3670,04/10/2021,13.0,Puerto Rico,2009.0


As we can see, Puerto Rico and the United States as a whole are added to this list. Let's remove them both.

In [27]:
df = df[df.State != 'Puerto Rico']
df = df[df.State != 'United States']
df.State.nunique(), df.State.unique()

(52,
 array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
        'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
        'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
        'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
        'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
        'New Jersey', 'New Mexico', 'New York', 'New York City',
        'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
        'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
        'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
        'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object))

We have 52 'states' here. We can see that we need to remove District of Columbia and New York City, too.

In [28]:
df = df[df.State != 'District of Columbia']
df = df[df.State != 'New York City']
df.State.nunique()

50

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3400 entries, 68 to 3603
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Week Ending Date    3400 non-null   object 
 1   COVID-19 Deaths     3400 non-null   float64
 2   State               3400 non-null   object 
 3   total_covid_deaths  3400 non-null   float64
dtypes: float64(2), object(2)
memory usage: 132.8+ KB


Now let's start adding features from our other dataframes

In [30]:
bea_gdp_raw.head()

,GeoFips,GeoName,LineCode,Description,2019:Q1,2019:Q2,2019:Q3,2019:Q4
0,0,United States,1,Real GDP (millions of chained 2012 dollars),1.895035e+07,1.902060e+07,1.914174e+07,1.925396e+07
1,0,United States,2,Chain-type quantity indexes for real GDP,1.169990e+02,1.174330e+02,1.181810e+02,1.188740e+02
2,0,United States,3,Current-dollar GDP (millions of current dollars),2.111531e+07,2.132988e+07,2.154032e+07,2.174739e+07
3,1000,Alabama,1,Real GDP (millions of chained 2012 dollars),1.998195e+05,2.000066e+05,2.014158e+05,2.020755e+05
4,1000,Alabama,2,Chain-type quantity indexes for real GDP,1.071110e+02,1.072110e+02,1.079670e+02,1.083200e+02


The yearly GDP for 2019 (which is what we'll be using) is calculated by averaging the 4 fiscal quarters. We'll make a new column for that. Next we only need the rows with the 'Description' 'Current-dollar GDP (millions of current dollars)'. We'll remove all the other rows.

In [31]:
df_gdp = bea_gdp_raw
df_gdp['GDP_millions'] = df_gdp[['2019:Q1', '2019:Q2', '2019:Q3', '2019:Q4']].mean(axis=1)
df_gdp.head()

,GeoFips,GeoName,LineCode,Description,2019:Q1,2019:Q2,2019:Q3,2019:Q4,GDP_millions
0,0,United States,1,Real GDP (millions of chained 2012 dollars),1.895035e+07,1.902060e+07,1.914174e+07,1.925396e+07,1.909166e+07
1,0,United States,2,Chain-type quantity indexes for real GDP,1.169990e+02,1.174330e+02,1.181810e+02,1.188740e+02,1.178717e+02
2,0,United States,3,Current-dollar GDP (millions of current dollars),2.111531e+07,2.132988e+07,2.154032e+07,2.174739e+07,2.143323e+07
3,1000,Alabama,1,Real GDP (millions of chained 2012 dollars),1.998195e+05,2.000066e+05,2.014158e+05,2.020755e+05,2.008293e+05
4,1000,Alabama,2,Chain-type quantity indexes for real GDP,1.071110e+02,1.072110e+02,1.079670e+02,1.083200e+02,1.076522e+02


In [32]:
df_gdp = df_gdp[df_gdp.Description == 'Current-dollar GDP (millions of current dollars)']
df_gdp.head()

,GeoFips,GeoName,LineCode,Description,2019:Q1,2019:Q2,2019:Q3,2019:Q4,GDP_millions
2,0,United States,3,Current-dollar GDP (millions of current dollars),21115309.0,21329877.0,21540325.0,21747394.0,2.143323e+07
5,1000,Alabama,3,Current-dollar GDP (millions of current dollars),225565.2,226927.0,229328.2,230750.1,2.281426e+05
8,2000,Alaska,3,Current-dollar GDP (millions of current dollars),54137.6,54279.9,54449.9,54674.7,5.438552e+04
11,4000,Arizona,3,Current-dollar GDP (millions of current dollars),361379.7,366904.9,373173.0,379018.8,3.701191e+05
14,5000,Arkansas,3,Current-dollar GDP (millions of current dollars),129192.6,130471.2,131556.2,132596.4,1.309541e+05


In [33]:
df_gdp.GeoName.unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'New England', 'Mideast',
       'Great Lakes', 'Plains', 'Southeast', 'Southwest',
       'Rocky Mountain', 'Far West'], dtype=object)

We have many areas that aren't states. Let's remove them.

In [34]:
df_gdp = df_gdp[df_gdp.GeoName != 'United States']
df_gdp = df_gdp[df_gdp.GeoName != 'District of Columbia']
df_gdp = df_gdp[df_gdp.GeoName != 'Mideast']
df_gdp = df_gdp[df_gdp.GeoName != 'Great Lakes']
df_gdp = df_gdp[df_gdp.GeoName != 'Plains']
df_gdp = df_gdp[df_gdp.GeoName != 'Southeast']
df_gdp = df_gdp[df_gdp.GeoName != 'Southwest']
df_gdp = df_gdp[df_gdp.GeoName != 'Rocky Mountain']
df_gdp = df_gdp[df_gdp.GeoName != 'Far West']
df_gdp = df_gdp[df_gdp.GeoName != 'New England']
df_gdp.reset_index(inplace=True)
df_gdp.GeoName.nunique()

50

Now let's take the desired columns

In [35]:
df_gdp = df_gdp[['GeoName', 'GDP_millions']]
df_gdp.head()

,GeoName,GDP_millions
0,Alabama,228142.625
1,Alaska,54385.525
2,Arizona,370119.100
3,Arkansas,130954.100
4,California,3132800.575


In [36]:
df = df.merge(df_gdp, left_on='State', right_on='GeoName')
df.drop(columns='GeoName', inplace=True, axis=1)
df.head()

,Week Ending Date,COVID-19 Deaths,State,total_covid_deaths,GDP_millions
0,01/04/2020,0.0,Alabama,0.0,228142.625
1,01/11/2020,0.0,Alabama,0.0,228142.625
2,01/18/2020,0.0,Alabama,0.0,228142.625
3,01/25/2020,0.0,Alabama,0.0,228142.625
4,02/01/2020,0.0,Alabama,0.0,228142.625


Time to add more variables

In [37]:
df_VO = cdc_VO_raw
df_VO.head()

,States,Republican,perc_pop_obese,pop_size
0,Alabama,1,36.1,4903
1,Alaska,1,30.5,731
2,Arizona,1,31.4,7278
3,Arkansas,1,37.4,3017
4,California,0,26.2,39512


In [38]:
df = df.merge(df_VO, left_on='State', right_on='States')
df.drop(columns='States', inplace=True, axis=1)
df.head()

,Week Ending Date,COVID-19 Deaths,State,total_covid_deaths,GDP_millions,Republican,perc_pop_obese,pop_size
0,01/04/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903
1,01/11/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903
2,01/18/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903
3,01/25/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903
4,02/01/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903


Time to add the percentage of the population of each state which lives in an urban environment. This data was collected as of 2010 so it is a little old, but should still work.

In [39]:
df_urb = cdc_urb_raw
df_urb.head()

,States,perc_pop_urban
0,Alabama,59.0
1,Alaska,66.0
2,Arizona,89.8
3,Arkansas,56.2
4,California,95.0


In [40]:
df = df.merge(df_urb, left_on='State', right_on='States')
df.drop(columns='States', inplace=True, axis=1)
df.head()

,Week Ending Date,COVID-19 Deaths,State,total_covid_deaths,GDP_millions,Republican,perc_pop_obese,pop_size,perc_pop_urban
0,01/04/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903,59.0
1,01/11/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903,59.0
2,01/18/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903,59.0
3,01/25/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903,59.0
4,02/01/2020,0.0,Alabama,0.0,228142.625,1,36.1,4903,59.0


Time to export the Data Frame for the Exploratory Data Analysis.

In [41]:
df.to_csv(r'C:\Users\agrae\Documents\AAA_Work\Coding\Springboard\HW\000_Capstone_2\Data\Clean_DF.csv')